In [2]:
import pandas as pd
df=pd.read_json("/content/problems_data_cleaned.jsonl",lines=True)

In [1]:
import pandas as pd
df_features=pd.read_csv("/content/metadata_features.csv")

In [3]:
import pandas as pd
words_df=pd.read_json("/content/no_stop_words.jsonl",lines=True)

In [4]:
import re
import pickle
import pandas as pd
import numpy as np

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import StandardScaler
from scipy.sparse import hstack


In [5]:
df.head(1)

,text,problem_level,problem_score
0,unununium was the name of the chemical eleme...,2,9.7


In [6]:
df_features.head(1)

,word_count,char_count,avg_word_len,unique_word_count,lexical_richness,num_density,topic_graph,topic_math,topic_strings,topic_optimization
0,286,1503,4.216783,147,0.512195,6,11,0,0,1


In [7]:
#table with no stop words
words_df.head(1)

,clean_text,problem_level,problem_score
0,unununium chemical element atom number nu...,2,9.7


In [10]:
tfidf = TfidfVectorizer(
    max_features=5000,
    ngram_range=(1, 3),
    min_df=5
)

X_text = tfidf.fit_transform(df["text"])

scaler = StandardScaler()
X_struct = scaler.fit_transform(df_features)


X_final = hstack([X_text, X_struct])

y = df["problem_level"]


In [11]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split


X = tfidf.fit_transform(df['text'].fillna(''))
y = df['problem_level']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=2022)
X.shape


(3899, 5000)

In [12]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC
from imblearn.over_sampling import SMOTE
from imblearn.combine import SMOTETomek
from sklearn.metrics import classification_report, confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt


In [13]:


rf_base = RandomForestClassifier(class_weight='balanced', random_state=2022)
rf_base.fit(X_train, y_train)
print(classification_report(y_test, rf_base.predict(X_test)))




              precision    recall  f1-score   support

           0       0.45      0.17      0.25       145
           1       0.36      0.13      0.19       266
           2       0.50      0.86      0.63       369

    accuracy                           0.48       780
   macro avg       0.44      0.39      0.36       780
weighted avg       0.44      0.48      0.41       780



In [16]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import StandardScaler
from scipy.sparse import hstack

# 1. Upgrade TF-IDF: Remove stopwords and add Bigrams
tfidf_v2 = TfidfVectorizer(
    max_features=3000,
    stop_words='english',
    ngram_range=(1, 2)  # Sees phrases like "shortest path"
)

text_vectors_v2 = tfidf_v2.fit_transform(df['text'])

# 2. Scale your Metadata (this was the missing link in your RF)
scaler = StandardScaler()
metadata_scaled = scaler.fit_transform(df_features)

# 3. Combine
X_v2 = hstack([text_vectors_v2, metadata_scaled])

# 4. Train a better model (Logistic Regression often beats RF for high-dimensional text)
from sklearn.linear_model import LogisticRegression

X_train, X_test, y_train, y_test = train_test_split(X_v2, y, test_size=0.2, stratify=y, random_state=42)

model_v2 = LogisticRegression(class_weight='balanced', max_iter=2000)
model_v2.fit(X_train, y_train)

print(classification_report(y_test, model_v2.predict(X_test)))

              precision    recall  f1-score   support

           0       0.45      0.64      0.53       145
           1       0.40      0.39      0.39       266
           2       0.63      0.53      0.58       369

    accuracy                           0.50       780
   macro avg       0.49      0.52      0.50       780
weighted avg       0.52      0.50      0.50       780



In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix
from scipy.sparse import hstack

# --- 1. PREPARE TEXT FEATURES (NGRAMS + STOPWORDS) ---
# We use ngram_range=(1, 2) to capture phrases like "shortest path"
# We remove 'english' stopwords to get rid of "the", "and", "of"
tfidf = TfidfVectorizer(
    max_features=5000,
    stop_words='english',
    ngram_range=(1, 2)
)
text_vectors = tfidf.fit_transform(df['text'])

# --- 2. SCALE METADATA FEATURES ---
# metadata_vectors comes from the features_df we created earlier (word_count, etc.)
scaler = StandardScaler()
metadata_scaled = scaler.fit_transform(df_features)

# --- 3. COMBINE EVERYTHING ---
# Combine the sparse TF-IDF matrix with the scaled numerical features
X_combined = hstack([text_vectors, metadata_scaled])
y = df['problem_level']

# --- 4. TRAIN/TEST SPLIT ---
X_train, X_test, y_train, y_test = train_test_split(
    X_combined, y, test_size=0.2, random_state=42, stratify=y
)

# --- 5. THE MODEL (LOGISTIC REGRESSION SCALED) ---
# We use class_weight='balanced' to fight the "Majority Bias"
# This tells the model to care more about the 727 Easy samples
lr_model = LogisticRegression(
    class_weight='balanced',
    max_iter=2000,
    C=1.0  # Regularization strength
)

lr_model.fit(X_train, y_train)
y_pred = lr_model.predict(X_test)

# --- 6. RESULTS ---
print("--- Final Model: Scaled Logistic Regression ---")
print(classification_report(y_test, y_pred))

In [14]:


X_train, X_test, y_train, y_test = train_test_split(X_final, y, test_size=0.2, stratify=y, random_state=2022)
X.shape

rf_base = RandomForestClassifier(class_weight='balanced', random_state=2022)
rf_base.fit(X_train, y_train)
print("Baseline:", classification_report(y_test, rf_base.predict(X_test)))



Baseline:               precision    recall  f1-score   support

           0       0.43      0.19      0.26       145
           1       0.33      0.13      0.19       266
           2       0.50      0.82      0.62       369

    accuracy                           0.47       780
   macro avg       0.42      0.38      0.35       780
weighted avg       0.43      0.47      0.40       780



In [ ]:
from sklearn.svm import LinearSVC
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt

# Assume X_final, y ready (your data)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=2022, stratify=y)

# LinearSVC (TF-IDF king)
svc_best = LinearSVC(
    class_weight='balanced',
    dual=False,  # Faster for high dims
    max_iter=3000,
    random_state=2022
)

svc_best.fit(X_train, y_train)
y_pred_svc = svc_best.predict(X_test)

print("🏆 LinearSVC Balanced:")
print(classification_report(y_test, y_pred_svc, zero_division=0))



🏆 LinearSVC Balanced:
              precision    recall  f1-score   support

           0       0.40      0.41      0.41       153
           1       0.37      0.34      0.36       281
           2       0.55      0.58      0.56       389

    accuracy                           0.47       823
   macro avg       0.44      0.44      0.44       823
weighted avg       0.46      0.47      0.46       823



In [ ]:
import joblib
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer

# Rebuild exact pipeline
pipe = Pipeline([
    ('tfidf', TfidfVectorizer(max_features=15000, min_df=3, ngram_range=(2,3))),  # Your settings
    ('rf', RandomForestClassifier(class_weight='balanced', n_estimators=200, random_state=2022))
])

# Train on FULL dataset
pipe.fit(df['text'], df['problem_level'])  # Replace 'clean_text'

# Save
joblib.dump(pipe, 'problem_classifier_57.pkl')


['problem_classifier_57.pkl']